In [1]:
from pandas import *
from numpy import *

In [2]:
hogares=read_csv('../conjunto_de_datos_enigh_ns_2020_csv/conjunto_de_datos_concentradohogar_enigh_2020_ns/conjunto_de_datos/conjunto_de_datos_concentradohogar_enigh_2020_ns.csv',low_memory=False)
poblacion=read_csv('../conjunto_de_datos_enigh_ns_2020_csv/conjunto_de_datos_poblacion_enigh_2020_ns/conjunto_de_datos/conjunto_de_datos_poblacion_enigh_2020_ns.csv',low_memory=False)
agro=read_csv('../conjunto_de_datos_enigh_ns_2020_csv/conjunto_de_datos_agro_enigh_2020_ns/conjunto_de_datos/conjunto_de_datos_agro_enigh_2020_ns.csv',low_memory=False)
UBICA=read_csv('../conjunto_de_datos_enigh_ns_2020_csv/conjunto_de_datos_concentradohogar_enigh_2020_ns/catalogos/ubica_geo.csv',encoding='utf8',low_memory=False)
CU=read_csv('CU_municipios.csv',encoding='latin1')
region=read_excel('Regionalizacion para la ENIGH.xls')
tamLoc=read_csv('../conjunto_de_datos_enigh_ns_2020_csv/conjunto_de_datos_concentradohogar_enigh_2020_ns/catalogos/tam_loc.csv',encoding='utf8')

In [3]:
tamLoc=tamLoc.rename(columns={'descripcion':'TamLoc'})

In [4]:
hogares=hogares.merge(tamLoc,on='tam_loc')

In [5]:
hogares=hogares.merge(UBICA,on='ubica_geo')
hogares=hogares.merge(region,on='desc_ent')


In [6]:
hogares['pob_aprox']=hogares['tot_integ']*hogares['factor']

In [7]:
dic1={'Redad':[]}
for i in range(len(poblacion)):
    if poblacion.edad[i]<16:
        dic1['Redad'].append('15 o menos')
    elif poblacion.edad[i]>=16 and poblacion.edad[i]<30:
        dic1['Redad'].append('16-29')
    elif poblacion.edad[i]>=30 and poblacion.edad[i]<60:
        dic1['Redad'].append('30-60')
    else:
        dic1['Redad'].append('60 o mas')

In [8]:
hogares=poblacion.join(DataFrame(dic1)).filter(['folioviv','foliohog','Redad','numren']).pivot_table(index=['folioviv','foliohog'],columns='Redad',aggfunc='count',margins=True,margins_name='Total').fillna(0).astype('int').drop(['Total'],axis=0).reset_index().merge(hogares,on=['folioviv','foliohog']).drop([('folioviv',''),('foliohog','')],axis=1).rename(columns={('numren', '15 o menos'):'15 o menos',('numren', '16-29'):'16-29',('numren', '30-60'):'30-59',('numren', '60 o mas'):'60 o mas',('numren','Total'):'numren'})

/usr/lib/python3/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [9]:
hogares['total_mujeres']=hogares['mujeres']*hogares['factor']
hogares['total_hombres']=hogares['hombres']*hogares['factor']
hogares['T_15 o menos']=hogares['15 o menos']*hogares['factor']
hogares['T_16-29']=hogares['16-29']*hogares['factor']
hogares['T_30-59']=hogares['30-59']*hogares['factor']
hogares['T_60 o mas']=hogares['60 o mas']*hogares['factor']
hogares['T_numren']=hogares['numren']*hogares['factor']

In [10]:
dic={'tipo_hogar':[]}
rurales=hogares.loc[hogares['tam_loc']==4]
rurales=rurales.reset_index()


In [11]:
for i in range(len(rurales)):
    if rurales['agrope'][i]>0 and rurales['noagrop'][i]==0:
        dic['tipo_hogar'].append('UCE')
    elif rurales['agrope'][i]>0 and rurales['noagrop'][i]>0:
        dic['tipo_hogar'].append('UCEP')
    elif rurales['agrope'][i]==0 and rurales['noagrop'][i]>0:
        dic['tipo_hogar'].append('UFR')
    else:
        dic['tipo_hogar'].append('Sin ingresos propios')

In [12]:
rurales=rurales.join(DataFrame(dic))

In [13]:
rurales=rurales.drop('index',axis=1)

In [14]:
autoconsumo=agro.filter(['folioviv','foliohog','autocons']).merge(rurales,on=['folioviv','foliohog'])

In [15]:
hogares.to_csv('hogares20_6_5_2022.csv',index=False)
rurales.to_csv('rurales20_6_5_2022.csv',index=False)
autoconsumo.to_csv('autoconsumo20_6_5_2022.csv',index=False)

In [16]:
cu=CU.filter(['CVE_MUN','LOC_0','LOC_1','LOC_2','LOC_3','LOC_4','LOC_TOTAL'])
cu=cu.astype('int')
cu['ploc_0']=round((cu['LOC_0']/cu['LOC_TOTAL'])*100,2)
cu['ploc_1']=round((cu['LOC_1']/cu['LOC_TOTAL'])*100,2)
cu['ploc_2']=round((cu['LOC_2']/cu['LOC_TOTAL'])*100,2)
cu['ploc_3']=round((cu['LOC_3']/cu['LOC_TOTAL'])*100,2)
cu['ploc_4']=round((cu['LOC_4']/cu['LOC_TOTAL'])*100,2)

In [17]:
cu=cu.rename(columns={'CVE_MUN':'ubica_geo'})

In [18]:
dic2={'pcu':[]}
for i in range(len(cu)):
    M=max(cu['ploc_0'][i],cu['ploc_1'][i],cu['ploc_2'][i],cu['ploc_3'][i],cu['ploc_4'][i])
    if cu['ploc_0'][i]==M:
        dic2['pcu'].append(0)
    elif cu['ploc_1'][i]==M:
        dic2['pcu'].append(1)
    elif cu['ploc_2'][i]==M:
        dic2['pcu'].append(2)
    elif cu['ploc_3'][i]==M:
        dic2['pcu'].append(3)
    else: 
        dic2['pcu'].append(4)
cu=cu.join(DataFrame(dic2))

In [19]:
cu

,ubica_geo,LOC_0,LOC_1,LOC_2,LOC_3,LOC_4,LOC_TOTAL,ploc_0,ploc_1,ploc_2,ploc_3,ploc_4,pcu
0,1001,6,279,29,233,42,589,1.02,47.37,4.92,39.56,7.13,1
1,1002,3,27,27,109,2,168,1.79,16.07,16.07,64.88,1.19,3
2,1003,2,52,7,60,34,155,1.29,33.55,4.52,38.71,21.94,3
3,1004,1,0,15,37,1,54,1.85,0.00,27.78,68.52,1.85,3
4,1005,6,142,16,61,10,235,2.55,60.43,6.81,25.96,4.26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,32054,1,0,2,37,8,48,2.08,0.00,4.17,77.08,16.67,3
2452,32055,2,0,11,47,52,112,1.79,0.00,9.82,41.96,46.43,4
2453,32056,1,25,0,40,2,68,1.47,36.76,0.00,58.82,2.94,3
2454,32057,1,0,4,14,0,19,5.26,0.00,21.05,73.68,0.00,3


In [20]:
hogares_cu=hogares.merge(cu,on='ubica_geo')
rurales_cu=rurales.merge(cu,on='ubica_geo')

In [21]:
hogares_cu

,folioviv,foliohog,15 o menos,16-29,30-59,60 o mas,numren,ubica_geo,tam_loc,est_socio,...,LOC_2,LOC_3,LOC_4,LOC_TOTAL,ploc_0,ploc_1,ploc_2,ploc_3,ploc_4,pcu
0,100013605,1,1,1,1,0,3,1001,1,3,...,29,233,42,589,1.02,47.37,4.92,39.56,7.13,1
1,100013606,1,1,1,2,0,4,1001,1,3,...,29,233,42,589,1.02,47.37,4.92,39.56,7.13,1
2,100017801,1,0,2,0,0,2,1001,1,3,...,29,233,42,589,1.02,47.37,4.92,39.56,7.13,1
3,100017802,1,1,1,0,0,2,1001,1,3,...,29,233,42,589,1.02,47.37,4.92,39.56,7.13,1
4,100017803,1,0,0,1,1,2,1001,1,3,...,29,233,42,589,1.02,47.37,4.92,39.56,7.13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77552,3260770714,1,0,0,0,2,2,32012,4,2,...,0,29,26,55,0.00,0.00,0.00,52.73,47.27,3
77553,3260770715,1,0,0,0,2,2,32012,4,2,...,0,29,26,55,0.00,0.00,0.00,52.73,47.27,3
77554,3260770716,1,1,0,2,0,3,32012,4,2,...,0,29,26,55,0.00,0.00,0.00,52.73,47.27,3
77555,3260770717,1,3,2,2,0,7,32012,4,2,...,0,29,26,55,0.00,0.00,0.00,52.73,47.27,3


In [22]:
rurales['TT_15 o menos']=round((rurales['T_15 o menos']/rurales['T_numren'])*rurales['pob_aprox'],2)
rurales['TT_16-29']=round((rurales['T_16-29']/rurales['T_numren'])*rurales['pob_aprox'],2)
rurales['TT_30-59']=round((rurales['T_30-59']/rurales['T_numren'])*rurales['pob_aprox'],2)
rurales['TT_60 o mas']=round((rurales['T_60 o mas']/rurales['T_numren'])*rurales['pob_aprox'],2)

In [23]:
hogares['TT_15 o menos']=(hogares['T_15 o menos']/hogares['T_numren'])*hogares['pob_aprox']
hogares['TT_16-29']=(hogares['T_16-29']/hogares['T_numren'])*hogares['pob_aprox']
hogares['TT_30-59']=(hogares['T_30-59']/hogares['T_numren'])*hogares['pob_aprox']
hogares['TT_60 o mas']=(hogares['T_60 o mas']/hogares['T_numren'])*hogares['pob_aprox']

In [24]:
hogares.to_csv('hogares20_6_5_2022.csv',index=False)
rurales.to_csv('rurales20_6_5_2022.csv',index=False)

In [25]:
rurales.keys().values

array(['folioviv', 'foliohog', '15 o menos', '16-29', '30-59', '60 o mas',
       'numren', 'ubica_geo', 'tam_loc', 'est_socio', 'est_dis', 'upm',
       'factor', 'clase_hog', 'sexo_jefe', 'edad_jefe', 'educa_jefe',
       'tot_integ', 'hombres', 'mujeres', 'mayores', 'menores', 'p12_64',
       'p65mas', 'ocupados', 'percep_ing', 'perc_ocupa', 'ing_cor',
       'ingtrab', 'trabajo', 'sueldos', 'horas_extr', 'comisiones',
       'aguinaldo', 'indemtrab', 'otra_rem', 'remu_espec', 'negocio',
       'noagrop', 'industria', 'comercio', 'servicios', 'agrope',
       'agricolas', 'pecuarios', 'reproducc', 'pesca', 'otros_trab',
       'rentas', 'utilidad', 'arrenda', 'transfer', 'jubilacion', 'becas',
       'donativos', 'remesas', 'bene_gob', 'transf_hog', 'trans_inst',
       'estim_alqu', 'otros_ing', 'gasto_mon', 'alimentos', 'ali_dentro',
       'cereales', 'carnes', 'pescado', 'leche', 'huevo', 'aceites',
       'tuberculo', 'verduras', 'frutas', 'azucar', 'cafe', 'especias',
       

In [26]:
rurales.tipo_hogar

0        Sin ingresos propios
1        Sin ingresos propios
2        Sin ingresos propios
3                         UFR
4                         UFR
                 ...         
30046    Sin ingresos propios
30047                     UFR
30048    Sin ingresos propios
30049    Sin ingresos propios
30050    Sin ingresos propios
Name: tipo_hogar, Length: 30051, dtype: object

In [28]:
linea=read_excel('lineaspobreza2020.xls')

In [30]:
linea['LPI rural'][0]

2485.63

In [31]:
LPIRURAL=[]
for i in linea['LPI rural']:
    LPIRURAL.append(i)

In [32]:
lpi_rural=round(array(LPIRURAL).mean(),2)

In [33]:
LPEIRURAL=[]
for i in linea['LPEI rural']:
    LPEIRURAL.append(i)

In [34]:
lpei_rural=round(array(LPEIRURAL).mean(),2)

In [35]:
sign((rurales['ing_cor'][0])/3-lpi_rural)

1.0

In [36]:
lpi={'LPI':[]}
for i in range(len(rurales)):
    if (rurales['ing_cor'][i])/3-lpi_rural > 0:
        lpi['LPI'].append('sobre')
    else:
        lpi['LPI'].append('bajo')
    

In [37]:
rurales=rurales.join(DataFrame(lpi))

In [38]:
lpei={'LPEI':[]}
for i in range(len(rurales)):
    if (rurales['ing_cor'][i])/3-lpei_rural > 0:
        lpei['LPEI'].append('sobre')
    else:
        lpei['LPEI'].append('bajo')

In [39]:
rurales=rurales.join(DataFrame(lpei))

In [40]:
rurales['tasa ocupacional']=round((rurales['ocupados']/rurales['tot_integ'])*100,2)

In [41]:
dic3={'tocupa':[]}
for i in range(len(rurales)):
    if rurales['tasa ocupacional'][i]<33.:
        dic3['tocupa'].append('Baja')
    elif rurales['tasa ocupacional'][i]>=33. and rurales['tasa ocupacional'][i]<66.:
        dic3['tocupa'].append('Media')
    elif rurales['tasa ocupacional'][i]>=66.:
        dic3['tocupa'].append('Alta')

In [42]:
rurales=rurales.join(DataFrame(dic3))

In [43]:
rurales.to_csv('rurales20_6_5_2022.csv',index=False)

In [44]:
T5=poblacion.filter(['folioviv', 'foliohog', 'sexo', 'etnia']).pivot_table(index=['folioviv','foliohog'],columns='etnia',aggfunc='count').fillna(0).astype('int').droplevel(level=0,axis=1).reset_index()

In [45]:
T5=T5.rename(columns={'1':'pertenece a etnia','2':'no pertenece a etnia'}).drop(' ' ,axis=1)

In [46]:
rurales=T5.merge(rurales, on=['folioviv','foliohog'])

In [47]:
cumun2=[]
for i in range(len(cu)):
    cumun2.append(max(cu['ploc_0'].loc[i],cu['ploc_1'].loc[i],cu['ploc_2'].loc[i],cu['ploc_3'].loc[i],cu['ploc_4'].loc[i]))

In [107]:
cu['CUmun2']=cumun2

In [108]:
cu['CUmun2']

0       47.37
1       64.88
2       38.71
3       68.52
4       60.43
        ...  
2451    77.08
2452    46.43
2453    58.82
2454    73.68
2455    71.43
Name: CUmun2, Length: 2456, dtype: float64

In [48]:
dic={'CUmun':[]}
for i in range(len(cu)):
    if cu['ploc_0'][i]>=60:
        dic['CUmun'].append(0)
    elif cu['ploc_1'][i]>=60:
        dic['CUmun'].append(1)
    elif cu['ploc_2'][i]>=60:
        dic['CUmun'].append(2)
    elif cu['ploc_3'][i]>=60:
        dic['CUmun'].append(3)
    elif cu['ploc_4'][i]>=60:
        dic['CUmun'].append(4)
    else:
        dic['CUmun'].append('NA')

In [49]:
cu=cu.join(DataFrame(dic))

In [50]:
dic2={'CUmun3':[]}
for i in range(len(cu)):
    if cu['ploc_0'][i]>=50:
        dic2['CUmun3'].append(0)
    elif cu['ploc_1'][i]>=50:
        dic2['CUmun3'].append(1)
    elif cu['ploc_2'][i]>=50:
        dic2['CUmun3'].append(2)
    elif cu['ploc_3'][i]>=50:
        dic2['CUmun3'].append(3)
    elif cu['ploc_4'][i]>=50:
        dic2['CUmun3'].append(4)
    else:
        dic2['CUmun3'].append('NA')

In [54]:
cu=cu.join(DataFrame(dic2))

ValueError: columns overlap but no suffix specified: Index(['CUmun3'], dtype='object')

In [53]:
cu=cu.drop('CUmun2',axis=1)

KeyError: "['CUmun2'] not found in axis"

In [55]:
dic3={'CUmun2':[]}
for i in range(len(cu)):
    if max(cu['ploc_0'][i],cu['ploc_1'][i],cu['ploc_2'][i],cu['ploc_3'][i],cu['ploc_4'][i])==cu['ploc_0'][i]:
        dic3['CUmun2'].append(0)
    elif max(cu['ploc_0'][i],cu['ploc_1'][i],cu['ploc_2'][i],cu['ploc_3'][i],cu['ploc_4'][i])==cu['ploc_1'][i]:
        dic3['CUmun2'].append(1)
    elif max(cu['ploc_0'][i],cu['ploc_1'][i],cu['ploc_2'][i],cu['ploc_3'][i],cu['ploc_4'][i])==cu['ploc_2'][i]:
        dic3['CUmun2'].append(2)
    elif max(cu['ploc_0'][i],cu['ploc_1'][i],cu['ploc_2'][i],cu['ploc_3'][i],cu['ploc_4'][i])==cu['ploc_3'][i]:
        dic3['CUmun2'].append(3)
    else:
        dic3['CUmun2'].append(4)

In [56]:
cu=cu.join(DataFrame(dic3))

In [57]:
rurales=rurales.merge(cu.filter(['ubica_geo','CUmun','CUmun2','CUmun3']),on='ubica_geo')

In [128]:
rurales.CUmun.loc[rurales['CUmun']!='NA']

219      3
220      3
221      3
222      3
223      3
        ..
29890    4
29891    4
29892    4
29893    4
29894    4
Name: CUmun, Length: 11111, dtype: object

In [61]:
rurales.to_csv('rurales20_6_5_2022.csv',index=False)

In [2]:
rurales=read_csv('rurales20_6_5_2022.csv')

In [59]:
LineasPobreza=[]
for i in range(len(rurales)):
    if rurales.LPEI[i]=='bajo':
        LineasPobreza.append('bajo')
    elif rurales.LPEI[i]=='sobre':
        if rurales.LPI[i]=='bajo':
            LineasPobreza.append('entre lineas')
        else:
            LineasPobreza.append('sobre')

In [60]:
rurales['LP']=LineasPobreza

In [17]:
rurales.filter(['LP','LPI','LPEI']).plot()

Matplotlib is building the font cache; this may take a moment.


TypeError: no numeric data to plot

In [18]:
rurales.to_csv('rurales20_6_5_2022.csv',index=False)